In [7]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import glob
import pandas as pd
import matplotlib 
from astropy.table import Table
import scipy.signal as sp

In [8]:
# define a funtion to return 0 when divided by zero
def div0( a, b, fill=0. ):
    """ a / b, divide by 0 -> `fill`
        div0( [-1, 0, 1], 0, fill=np.nan) -> [nan nan nan]
        div0( 1, 0, fill=np.inf ) -> inf
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide( a, b )
    if np.isscalar( c ):
        return c if np.isfinite( c ) \
            else fill
    else:
        c[ ~ np.isfinite( c )] = fill
        return c

def normalize(data):
    return ((data - np.min(data)) / (np.max(data) - np.min(data))) + 1


def df2array(df):
    lists = []
    for i in range(len(df)):
        lists.append(df.iloc[i][0])
    return np.array(lists)


# define a function that resamples the data based on wavelength
# has 3 1darray inputs: target_array is the array that has the sampling you want;
# ref_array is the array that has the same kind of data with the target_array but with different sampling rate:
# and the input_array is the array sharing the same sampling rate but not necessicirally the same data with the ref_array

# this function resamlpes the ref_array to the same sampling as the target_array first and then
# use the indecies to resample the input array, output is the resampled input_array


def resampling(target_array, ref_array, input_array, resamp_ref_array=False):

    # for each value in the target_array, find the indeices of the value(s)
    # nearest to it in the ref_array, output has the same legth as the target_array
    index = abs(target_array[:, None] - ref_array[None, :]).argmin(axis=-1)

    # based on the indices find the corresponding values in the input_array and
    # write everything into a new array
    resampled_array = []
    for i in index:
        resampled_array.append(input_array[i])

    # choose if to output the resampled ref_array, sometimes you need this
    if resamp_ref_array == True:
        resampled_ref_array = []
        for i in index:
            resampled_ref_array.append(ref_array[i])
        return resampled_array, resampled_ref_array
    else:
        return resampled_array


def find_files(date, name):

    spec1d_list = glob.glob('NIRES_PypeIt/' + date + '_reduced' + '/Science/' +
                            'spec1d*.fits')
    target_files = []
    for i in range(len(spec1d_list)):
        if name in spec1d_list[i]:
            target_files.append(spec1d_list[i])
    return target_files

def stack_1order(frame_list, order):
    all_flux = []
    all_ivar = []
    all_wave = []
    for frame_id in range(len(frame_list)):
        hdul = fits.open(frame_list[frame_id])
        data = hdul[order].data
        all_flux.append(data['OPT_COUNTS'])
        all_ivar.append(data['OPT_COUNTS_IVAR'])
        all_wave.append(data['OPT_WAVE'])
        hdul.close()
    stacked_flux = np.median(all_flux, axis=0)
    
    stacked_var  = np.sum(div0(1,all_ivar),axis=0)/np.square(len(all_ivar[0]))
    stacked_ivar = div0(1,stacked_var)
    stacked_wave = np.median(all_wave, axis=0)

    stacked_data = [stacked_wave, stacked_flux, stacked_ivar]
    return stacked_data

### Read the model and sci data

In [9]:
# specifiy which date to work with
date = '20180630'

# read the model data
A0_model = pd.read_csv('NIRES_PypeIt/A0V.dat',
                       skiprows=2,
                       header=None,
                       names=['lk', 'ukf_a0v', 'uks_a0v', 'fh', 'fl', 'fc'],
                       sep='  ',
                       engine='python')

A0_flux = df2array(A0_model[['ukf_a0v']])
A0_wave = df2array(A0_model[['lk']])

sci_files = find_files(date, 'J2202-0033')

### Preform Telluric Correction Pre Slit

In [10]:
# read the telluric data and airmass
tell_files = find_files(date, 'HIP')

tell_airmass_list = []
for frame_id in range(len(tell_files)):
    hdul = fits.open(tell_files[frame_id])
    hdr = hdul[0].header
    tell_airmass_list.append(hdr['AIRMASS'])
    hdul.close()
tell_airmass_list = np.array(tell_airmass_list)

In [11]:
def tellcorr_preslit(sci_file, peakcoord_offset=4, rel_height=0.85):
    peak_coords = [9546, 10050, 10940, 12685, 12820]
    sci_hdul = fits.open(sci_file)
    sci_airmass = sci_hdul[0].header['AIRMASS']

    # caculate the airmass difference
    airmass_diff = tell_airmass_list - sci_airmass

    # determine which telluric frame(s) to use by airmass
    # if the differences are all positive, i.e. all tell airmass are
    # greater than the sci airmass
    if min(airmass_diff >= 0):
        tell1_id = airmass_diff.argmin()
        # path or filename
        tell1_path = tell_files[tell1_id]
        tell2_path = 'None'
    # if all negative, i.e. all tell airmass smaller than the sci airmass
    elif max(airmass_diff < 0):
        tell1_id = airmass_diff.argmax()
        # path or filename
        tell1_path = tell_files[tell1_id]
        tell2_path = 'None'
    # when sci airmass is in between two tellairmass
    else:
        # tell1 is the frame with greater airmass
        # tell2 is the frame with smaller airmass
        tell1_id = airmass_diff.index(min(i for i in airmass_diff if i > 0))
        tell2_id = airmass_diff.index(min(i for i in airmass_diff if i < 0))
        tell1_airmass = tell_airmass_list[tell1_id]
        tell2_airmass = tell_airmass_list[tell2_id]
        # weight of the two telluric frames based on
        # airmass relative to science airmass
        weight1 = abs(tell2_airmass - sci_airmass)
        weight2 = abs(tell1_airmass - sci_airmass)
        # path or file name
        tell1_path = tell_files[tell1_id]
        tell2_path = tell_files[tell2_id]

    ############################################################################
    # start the mean process
    for order in range(1, 6):
        # define the science and telluric wave and flux
        sci_data = sci_hdul[order].data

        sci_wave, sci_flux = sci_data['OPT_WAVE'], sci_data['OPT_COUNTS']
        sci_ivar = sci_data['OPT_COUNTS_IVAR']
        sci_sig = sci_data['OPT_COUNTS_SIG']

        # if the airmass of the sci frame is not in between two telluric arimasses,
        # then just use one telluric
        if tell2_path == 'None':
            tell_hdul = fits.open(tell1_path)
            tell_data = tell_hdul[order].data
            tell_wave, tell_flux = tell_data['OPT_WAVE'], tell_data[
                'OPT_COUNTS']
            tell_hdul.close()
            # resample telluric data to science data
            tell_flux = resampling(sci_wave, tell_wave, tell_flux)

        # if the airmass of the sci frame is in between two telluric arimasses,
        # then caculate the weighted mean of the two telluric flux
        else:
            tell1_hdul = fits.open(tell1_path)
            tell2_hdul = fits.open(tell2_path)
            tell1_wave, tell1_flux = tell1_hdul[order].data[
                'OPT_WAVE'], tell1_hdul[order].data['OPT_COUNTS']
            tell2_wave, tell2_flux = tell2_hdul[order].data[
                'OPT_WAVE'], tell2_hdul[order].data['OPT_COUNTS']
            tell1_hdul.close()
            tell2_hdul.close()
            # resample telluric data to science data
            tell1_flux = resampling(sci_wave, tell1_wave, tell1_flux)
            tell2_flux = resampling(sci_wave, tell2_wave, tell2_flux)
            tell_flux = (tell1_flux * weight1 +
                         tell2_flux * weight2) / (weight1 + weight2)

        # slice model wavelength to make it in the same range
        # as the wavelenghth range of the working order
        A0_lo = np.abs(A0_wave - sci_wave[0]).argmin()
        A0_up = np.abs(A0_wave - sci_wave[-1]).argmin()
        A0_wave_sliced = A0_wave[A0_lo:A0_up]
        A0_flux_sliced = A0_flux[A0_lo:A0_up]
        # resample the model data to the same rate as the science data
        A0_flux_sliced = resampling(
            sci_wave,
            A0_wave_sliced,
            A0_flux_sliced,
        )

        # define the telluric correction factor
        tellcorr_factor = div0(A0_flux_sliced, tell_flux)

        # interpolate the pachen series peaks
        for peak_coord in peak_coords:
            # if the wavelength is in the range of the order
            if peak_coord >= sci_wave[0] and peak_coord <= sci_wave[-1]:
                # find the x index of the peak
                # there might be an offset so check the near values to find the real peak
                # the offset might be changed
                given_peak_id = abs(peak_coord - sci_wave).argmin()
                peak_range_id = np.linspace(given_peak_id - peakcoord_offset,
                                            given_peak_id + peakcoord_offset,
                                            2 * peakcoord_offset + 1,
                                            dtype=int)
                peak_range_values = []
                for i in peak_range_id:
                    peak_range_values.append(tellcorr_factor[i])
                # use the real peak value to find the index
                peak_id = [peak_range_id[np.array(peak_range_values).argmax()]]
                peak_width = sp.peak_widths(tellcorr_factor, peak_id,
                                            rel_height)[0]
                # find the index of the right and left limit of the peak
                # assume symmetry
                right_id = peak_id[0] + round(peak_width[0] / 2)
                left_id = peak_id[0] - round(peak_width[0] / 2)
                width = right_id - left_id
                # the left and right x indices around the peak,
                # used to interpolate new values for the peak region
                xp = np.hstack((np.linspace(left_id - width,
                                            left_id - 1,
                                            width,
                                            dtype=int),
                                np.linspace(right_id + 1,
                                            right_id + width,
                                            width,
                                            dtype=int)))
                # the corresponding y values
                fp = []
                for i in xp:
                    fp.append(tellcorr_factor[i])
                # interpolate range (the width of the peak)
                x_interp = np.linspace(left_id, right_id, width + 1, dtype=int)
                # interpolate
                interp_y = np.interp(x_interp, xp, fp)
                # rewrite and replace
                for i in range(len(x_interp)):
                    tellcorr_factor[x_interp[i]] = interp_y[i]
            else:
                pass

        # perform the correction
        tellcorr_sci = np.multiply(sci_flux, tellcorr_factor)

        # rescale (not propograte) the error
        tellcorr_sci_sig = np.multiply(sci_sig, div0(A0_flux_sliced,
                                                     tell_flux))

        # save the result to over write the origional data
        sci_hdul[order].data['OPT_COUNTS'] = tellcorr_sci
        sci_hdul[order].data['OPT_COUNTS_IVAR'] = div0(
            1, np.square(tellcorr_sci_sig))
        sci_hdul[order].data['OPT_COUNTS_SIG'] = tellcorr_sci_sig

    return sci_hdul.writeto('NIRES_PypeIt/' + date + '_reduced/' +
                            'tellcorr_science/' + sci_file[38:44] +
                            '_tellcorr' + sci_file[44:],
                            overwrite=True)

In [12]:
for sci_file in sci_files:
    tellcorr_preslit(sci_file)